<a href="https://colab.research.google.com/github/namoshi/dl_intro/blob/master/lab_tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2.Auto Encoder

AutoEncoder generally performs learning to minimize mean squared errors between input and output.

Thus, the output of the AutoEncoder is the same as the input.

The features of the hidden layer are applied to various tasks.

※If you want to perform learning, create a source file.

Import of required library.

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
from skimage import io

Fixed seed value for random numbers.

In [0]:
torch.cuda.manual_seed_all(100100)
torch.manual_seed(100100)
np.random.seed(100100)

Parameter definition.

In [0]:
BATCH_SIZE      = 1000
WEIGHT_DECAY    = 0.001
LEARNING_RATE   = 0.01
EPOCH           = 100
NUM_WORKERS     = 2

Reading dataset.

In [0]:
mnist_train = dsets.MNIST(".", download=True, train=True)
dataTrain = []
dataTest = []


mnist_test = dsets.MNIST(".", download=True, train=False)



for i in range(len(mnist_train)):
	dataTrain.append(np.array(mnist_train[i][0]))
  

for i in range(len(mnist_test)):
	dataTest.append(np.array(mnist_test[i][0]))

#Change shape to (N, C, H, W) by reshape.
dataTrain = np.array(dataTrain)
dataTrain = dataTrain.reshape(len(mnist_train), 1, 28, 28)
dataTrain = dataTrain / 255.0

dataTest = np.array(dataTest)
dataTest = dataTest.reshape(len(mnist_test), 1, 28, 28)
dataTest = dataTest / 255.0

datasizeTrain = len(dataTrain)
datasizeTest  = len(dataTest)

Define the model structure.

In this example, a multi-layer perceptron having 10 units in hidden layer is used.

In [0]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(28*28*1, 10)
        self.fc2 = nn.Linear(10, 28*28*1)

    def forward(self, x):
        x = x.view(-1, 28*28*1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = x.view(BATCH_SIZE, 1, 28, 28)
        return x
      

net = Network()
net.cuda()

Define loss and optimizer.

In [0]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=WEIGHT_DECAY)

Input data into the model for each mini-batch and perform parameter update.

In [0]:
trainLoss = []
testLoss  = []


for epoch in range(1, EPOCH+1):
    print("epoch", epoch, "iter", epoch*(datasizeTrain//BATCH_SIZE))
    net.train()
    for i in range(0, datasizeTrain, BATCH_SIZE):
        perm = np.random.permutation(datasizeTrain)
        xBatch = dataTrain[perm[i:i+BATCH_SIZE]]
        tBatch = dataTrain[perm[i:i+BATCH_SIZE]]
        xBatch = torch.from_numpy(xBatch.copy()).float().cuda()
        tBatch = torch.from_numpy(tBatch.copy()).float().cuda()
        optimizer.zero_grad()
        yBatch = net(xBatch)
        loss = criterion(yBatch, tBatch)
        loss.backward()
        optimizer.step()

    net.eval()
    sumTrainLoss = 0.0
    with torch.no_grad():
        for i in range(0, datasizeTrain, BATCH_SIZE):
            perm = np.random.permutation(datasizeTrain)
            xBatch = dataTrain[perm[i:i+BATCH_SIZE]]
            tBatch = dataTrain[perm[i:i+BATCH_SIZE]]
            xBatch = torch.from_numpy(xBatch.copy()).float().cuda()
            tBatch = torch.from_numpy(tBatch.copy()).float().cuda()
            yBatch = net(xBatch)
            loss = criterion(yBatch, tBatch)
            sumTrainLoss += float(loss.cpu().data.item()) * BATCH_SIZE
        trainLoss.append(sumTrainLoss / datasizeTrain)

        sumTestLoss = 0.0
        for i in range(0, datasizeTest, BATCH_SIZE):
            perm = np.random.permutation(datasizeTest)
            xBatch = dataTest[perm[i:i+BATCH_SIZE]] # (BATCH_SIZE, 1, 128, 128)
            tBatch = dataTest[perm[i:i+BATCH_SIZE]] # (BATCH_SIZE, 1, 128, 128)
            xBatch = torch.from_numpy(xBatch.copy()).float().cuda()
            tBatch = torch.from_numpy(tBatch.copy()).float().cuda()
            yBatch = net(xBatch)
            loss = criterion(yBatch, tBatch)
            sumTestLoss += float(loss.cpu().data.item()) * BATCH_SIZE
        testLoss.append(sumTestLoss / datasizeTest)

    print("train mean loss={}".format(sumTrainLoss / datasizeTrain))
    print("test  mean loss={}".format(sumTestLoss / datasizeTest))

Draw loss as a graph.

In [0]:
plt.figure(figsize=(6,6))

plt.plot(range(EPOCH), trainLoss)
plt.plot(range(EPOCH), testLoss)
plt.legend(['train loss', 'test loss'])
plt.title('loss')
plt.savefig("loss.png")

Save the output result of the network as an image.

In [0]:
result = np.zeros((datasizeTrain, 1, 28, 28))
result_test = np.zeros((datasizeTest, 1, 28, 28))

#Get network output for train data.
for i in range(0, datasizeTrain, BATCH_SIZE):
	xBatch = dataTrain[i:i+BATCH_SIZE]
	xBatch = torch.from_numpy(xBatch.copy()).float().cuda()
	re = net(xBatch)
	result[i:i+BATCH_SIZE] = re.cpu().data

#Get network output for test data.
for i in range(0, datasizeTest, BATCH_SIZE):
	xBatch = dataTest[i:i+BATCH_SIZE]
	xBatch = torch.from_numpy(xBatch.copy()).float().cuda()
	re = net(xBatch)
	result_test[i:i+BATCH_SIZE] = re.cpu().dat


  
#Save the image.
for i in range(datasizeTrain):
	pic = result[i]
	pic= pic.reshape((28,28))
	io.imsave("./imgtrain%d.png"%i, pic)


for i in range(datasizeTest):
	pic = result_test[i]
	pic= pic.reshape((28,28))
	io.imsave("./imgtest%d.png"%i, pic)
